In [2]:
import pandas as pd
from datetime import datetime, timedelta

file_name = 'aqi.csv'
try:
    df = pd.read_csv(file_name)
    print(f"Data loaded successfully from '{file_name}'.")
except FileNotFoundError:
    print(f"Error: The file '{file_name}' was not found. Please ensure it's in the same directory as your script, or provide the full path.")
    exit()

# --- Step 3: Convert the 'date' column to datetime objects ---
try:
    df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
    print("The 'date' column has been successfully converted to datetime objects.")
except KeyError:
    print("Error: 'date' column not found. Please check the column name in your data.")
    exit()
except Exception as e:
    print(f"Error converting 'date' column: {e}")
    print("Please ensure the 'date' column's format is consistently DD-MM-YYYY.")
    exit()

# --- Step 4: Convert 'aqi_value' to numeric (if it's not already) ---
try:
    original_dtype_aqi = df['aqi_value'].dtype
    if original_dtype_aqi == 'object':
        print(f"'{original_dtype_aqi}' dtype detected for 'aqi_value'. Attempting conversion to numeric.")
        df['aqi_value'] = pd.to_numeric(df['aqi_value'], errors='coerce')
        if df['aqi_value'].isnull().any():
            print("Warning: NaN values introduced in 'aqi_value' after conversion for 'aqi_value'.")
        else:
            print("Conversion successful: 'aqi_value' is now numeric with no new NaNs.")
    else:
        print(f"'aqi_value' is already of dtype: {original_dtype_aqi}. No conversion needed.")
except KeyError:
    print("Error: 'aqi_value' column not found. Please check the column name in your data.")
    exit()

# --- Step 5: Analysis: AQI on Weekends vs Weekdays in Indian Metro Cities (Last 1 Year) ---

print("\n--- Analyzing AQI on Weekends vs Weekdays in Indian Metro Cities (Last 1 Year) ---")

# Define Indian Metro Cities (Adjust names if they differ in your 'area' column)
# It's good practice to convert your 'area' column to title case if its casing is inconsistent.
# df['area'] = df['area'].str.title() # Uncomment if area names have inconsistent casing

metro_cities = [
    'Delhi',
    'Mumbai',
    'Chennai',
    'Kolkata',
    'Bengaluru', # Note: 'Bengaluru' is the official name, ensure your data uses it or 'Bangalore'
    'Hyderabad',
    'Ahmedabad',
    'Pune'
]

# Define the date range for "last 1 year" from today (July 26, 2025)
end_date = datetime(2025, 7, 25) # Up to yesterday (July 25, 2025)
start_date = end_date - timedelta(days=365) # One year prior

print(f"Considering data from {start_date.strftime('%d-%m-%Y')} to {end_date.strftime('%d-%m-%Y')}.")

# Filter data for metro cities and the last 1 year
filtered_metro_df = df[
    (df['area'].isin(metro_cities)) &
    (df['date'] >= start_date) &
    (df['date'] <= end_date)
].copy()

if filtered_metro_df.empty:
    print("\nNo data found for the specified metro cities in the last 1 year.")
    print("Please check city names in your 'area' column and the date range of your data.")
else:
    # Create a 'day_type' column (Weekday or Weekend)
    # Monday=0, Sunday=6
    filtered_metro_df['day_of_week'] = filtered_metro_df['date'].dt.dayofweek
    filtered_metro_df['day_type'] = filtered_metro_df['day_of_week'].apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')

    # Calculate average AQI for each city by day type
    aqi_comparison = filtered_metro_df.groupby(['area', 'day_type'])['aqi_value'].mean().unstack()

    print("\n--- Average AQI: Weekends vs. Weekdays in Metro Cities (Last 1 Year) ---")
    print(aqi_comparison.round(2)) # Round to 2 decimal places for readability

    # Optional: Calculate the difference or ratio for easier comparison
    if 'Weekday' in aqi_comparison.columns and 'Weekend' in aqi_comparison.columns:
        aqi_comparison['Difference (Weekday - Weekend)'] = aqi_comparison['Weekday'] - aqi_comparison['Weekend']
        aqi_comparison['Weekend vs Weekday Ratio'] = aqi_comparison['Weekend'] / aqi_comparison['Weekday']
        print("\n--- Comparison of Average AQI ---")
        print(aqi_comparison[['Weekday', 'Weekend', 'Difference (Weekday - Weekend)', 'Weekend vs Weekday Ratio']].round(2))

        print("\nInterpretation Guide:")
        print("  - A positive 'Difference (Weekday - Weekend)' means AQI is typically higher on weekdays.")
        print("  - A 'Weekend vs Weekday Ratio' less than 1 means AQI is typically lower on weekends (i.e., improvement).")

    else:
        print("\nCould not calculate difference/ratio as not all day types found for all cities.")

print("\n--- Analysis Complete ---")

Data loaded successfully from 'aqi.csv'.
The 'date' column has been successfully converted to datetime objects.
'aqi_value' is already of dtype: int64. No conversion needed.

--- Analyzing AQI on Weekends vs Weekdays in Indian Metro Cities (Last 1 Year) ---
Considering data from 25-07-2024 to 25-07-2025.

--- Average AQI: Weekends vs. Weekdays in Metro Cities (Last 1 Year) ---
day_type   Weekday  Weekend
area                       
Ahmedabad   117.23   120.11
Bengaluru    76.83    76.82
Chennai      73.07    68.54
Delhi       220.51   206.34
Hyderabad    82.98    83.12
Kolkata     103.73   103.32
Mumbai       99.97   102.56
Pune        110.56   109.39

--- Comparison of Average AQI ---
day_type   Weekday  Weekend  Difference (Weekday - Weekend)  \
area                                                          
Ahmedabad   117.23   120.11                           -2.88   
Bengaluru    76.83    76.82                            0.01   
Chennai      73.07    68.54                          

In [7]:
import pandas as pd
from datetime import datetime

file_name = 'aqi.csv'
df = pd.read_csv(file_name)
print(f"Data loaded successfully from '{file_name}'.")

try:
    df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
    print("The 'date' column has been successfully converted to datetime objects.")
except KeyError:
    print("Error: 'date' column not found. Please check the column name in your data.")
    exit()
except Exception as e:
    print(f"Error converting 'date' column: {e}")
    print("Please ensure the 'date' column's format is consistently DD-MM-YYYY.")
    exit()

f_area='Bengaluru'

# Define the date range from march to may ( 2025)
end_date = datetime(2025, 4, 30) # Up to yesterday (July 25, 2025)
start_date = datetime(2025, 3, 1)

print(f"considering date from {start_date}  to {end_date}")

filtered_df = df[
    (df['area']==f_area) &
    (df['date'] >= start_date) &
    (df['date'] <= end_date)
].copy()

# Create a new DataFrame that contains the dominant status per day (e.g., the most frequent one)
dominant_status = (
    filtered_df.groupby(['date'])['air_quality_status']
    .agg(lambda x: x.mode()[0])  # Use mode to get the most frequent status
)

# Now count how many days had each status
category_day_counts = dominant_status.value_counts().reset_index()
category_day_counts.columns = ['Air Quality Category', 'Number of Days']

print(category_day_counts)


Data loaded successfully from 'aqi.csv'.
The 'date' column has been successfully converted to datetime objects.
considering date from 2025-03-01 00:00:00  to 2025-04-30 00:00:00
  Air Quality Category  Number of Days
0         Satisfactory              48
1             Moderate              13


In [7]:
import pandas as pd

import numpy as np

file_name = 'vahan.csv'
df = pd.read_csv(file_name)
print(f"Data loaded successfully from '{file_name}'.")


df['EV or NonEV'] = np.where(
    df['fuel'].isin(["ELECTRIC(BOV)", "PURE EV", "STRONG HYBRID EV", "PLUG-IN HYBRID EV"]) |
    df['vehicle_class'].isin(["E-RICKSHAW(P)", "E-RICKSHAW WITH CART (G)"]),
    "EV",
    "NonEV"
)

print("Created 'EV or NonEV' columns.")
ev_df = df[df['EV or NonEV'] == 'EV']
top_ev_states = (
    ev_df.groupby('state')
    .size()
    .sort_values(ascending=False)
    .head(5)
    .index
    .tolist()
)
print("Top 5 states with highest EV adoption:", top_ev_states)

Data loaded successfully from 'vahan.csv'.
Created 'EV or NonEV' columns.
Top 5 states with highest EV adoption: ['Karnataka', 'Maharashtra', 'Uttar Pradesh', 'Rajasthan', 'Tamil Nadu']


In [8]:
df['ev_adoption_group']=df['state'].apply(lambda x: "High EV" if x in top_ev_states else "Low EV" )

In [16]:
import pandas as pd

import numpy as np

file_name = 'vahan.csv'
df = pd.read_csv(file_name)
print(f"Data loaded successfully from '{file_name}'.")
aqi_df = pd.read_csv('aqi.csv')


df['EV or NonEV'] = np.where(
    df['fuel'].isin(["ELECTRIC(BOV)", "PURE EV", "STRONG HYBRID EV", "PLUG-IN HYBRID EV"]) |
    df['vehicle_class'].isin(["E-RICKSHAW(P)", "E-RICKSHAW WITH CART (G)"]),
    "EV",
    "NonEV"
)

print("Created 'EV or NonEV' columns.")
ev_df = df[df['EV or NonEV'] == 'EV']
top_ev_states = (
    ev_df.groupby('state')
    .size()
    .sort_values(ascending=False)
    .head(5)
    .index
    .tolist()
)
print("Top 5 states with highest EV adoption:", top_ev_states)
aqi_df['ev_adoption_group']=aqi_df['state'].apply(lambda x: "High EV" if x in top_ev_states else "Low EV" )

avg_aqi_by_ev_group = aqi_df.groupby('ev_adoption_group')['aqi_value'].mean()

print("Average aqi_value is",avg_aqi_by_ev_group)

Data loaded successfully from 'vahan.csv'.
Created 'EV or NonEV' columns.
Top 5 states with highest EV adoption: ['Karnataka', 'Maharashtra', 'Uttar Pradesh', 'Rajasthan', 'Tamil Nadu']
Average aqi_value is ev_adoption_group
High EV    101.266680
Low EV     119.194758
Name: aqi_value, dtype: float64


In [19]:
from scipy.stats import ttest_ind

high_ev_aqi = aqi_df[aqi_df['ev_adoption_group'] == 'High EV']['aqi_value']
low_ev_aqi = aqi_df[aqi_df['ev_adoption_group'] == 'Low EV']['aqi_value']

t_stat, p_val = ttest_ind(high_ev_aqi, low_ev_aqi, equal_var=False)
print(f"\nT-Test: t-statistic = {t_stat:.2f}, p-value = {p_val:.4f}")



T-Test: t-statistic = -62.28, p-value = 0.0000


In [ ]:
import pandas as pd

import numpy as np

file_name = 'vahan.csv'
df = pd.read_csv(file_name)
print(f"Data loaded successfully from '{file_name}'.")
aqi_df = pd.read_csv('aqi.csv')

In [2]:
import pandas as pd
file_name = 'aqi.csv'

# --- Step 2: Load the Data ---
try:
    df = pd.read_csv(file_name)
    print(f"Data loaded successfully from {file_name}.")
except FileNotFoundError:
    print(f"Error: '{file_name}' not found. Please ensure the file is in the correct directory.")
    exit()

# --- Step 3: Convert the 'date' column to datetime objects ---
try:
    df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
    print("The 'date' column has been successfully converted to datetime objects.")
except Exception as e:
    print(f"Error converting 'date' column: {e}")
    print("Please check if the 'date' column truly follows the DD-MM-YYYY format or if its name is correct.")
    exit()

# --- Step 4: Convert 'aqi_value' to numeric (if it's not already) ---
# This ensures it's treated as a number in Power BI for calculations.
original_dtype_aqi = df['aqi_value'].dtype
if original_dtype_aqi == 'object':
    print(f"'{original_dtype_aqi}' dtype detected for 'aqi_value'. Attempting conversion to numeric.")
    df['aqi_value'] = pd.to_numeric(df['aqi_value'], errors='coerce')
    if df['aqi_value'].isnull().any():
        print("Warning: NaN values introduced in 'aqi_value' after conversion. Please inspect if unexpected.")
    else:
        print("Conversion successful: 'aqi_value' is now numeric with no new NaNs.")
else:
    print(f"'aqi_value' is already of dtype: {original_dtype_aqi}. No conversion needed.")

# --- Step 5: Create new time-based features (year, month, month_name) ---
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.month_name()
print("Created 'year', 'month', and 'month_name' columns.")


# --- Step 6: Export the main processed DataFrame to a CSV file for Power BI ---
output_file_name = 'air_quality_processed_for_powerbi.csv'
df.to_csv(output_file_name, index=False) # index=False prevents writing the Pandas DataFrame index as a column

print(f"\n--- Export Complete ---")
print(f"Your main processed DataFrame has been saved to: '{output_file_name}'")
print(f"You can now import this CSV file into Power BI Desktop.")

Data loaded successfully from aqi.csv.
The 'date' column has been successfully converted to datetime objects.
'aqi_value' is already of dtype: int64. No conversion needed.
Created 'year', 'month', and 'month_name' columns.

--- Export Complete ---
Your main processed DataFrame has been saved to: 'air_quality_processed_for_powerbi.csv'
You can now import this CSV file into Power BI Desktop.
